<a href="https://colab.research.google.com/github/sk27110/AutoPriceForecast/blob/main/auto_ru_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests as r
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import pandas as pd
import sys
import time
import datetime
import requests as r
import json
from bs4 import BeautifulSoup as bs
from pandas import Series
import requests as r, json
import re
from tqdm import tqdm


BASE_URL = 'https://auto.ru/cars/{year}-year/used/?displacement{vol}'
max_pages = 100
volumes = ['_from=2000', '_to=2000']


for year in range(1990, 2025):
  cars = []
  for page_number in tqdm(range(1, 101), desc=f'Страницы {year}', leave=False):
    for vol in volumes:
      try:
        url = BASE_URL.format(year=year, vol=vol) + f"&page={page_number}"
        response = r.get(url)
        response.encoding = 'utf-8'
        if response.status_code == 200:
          page = bs(response.text, 'html.parser')
          items = page.find_all(class_='ListingItem')
          for item in items:
            car = dict()
            car['price'] = item.find('span').text
            car['title'] = item.find(class_='Link ListingItemTitle__link').text
            car['year'] = item.find(class_='ListingItem__year').text
            car['mileage'] = result = re.sub(r'[^\dкм]', ' ', item.find(class_='ListingItem__kmAge').text).replace(' ', '').strip()
            data = item.find_all(class_= 'ListingItemTechSummaryDesktop__cell')
            car_tech_info = ['engine', 'transmission', 'body_type', 'drive_type', 'color']
            for x,y  in zip(car_tech_info, data):
              car[x]=y.text
            car['engine'] = car['engine'].replace('\u2009', ' ').replace('\xa0', ' ')
            cars.append(car)
        else:
          print(f"Ошибка при загрузке страницы {url}: {response.status_code}")
      except:
        print("Ошибка")
  df = pd.DataFrame(cars)
  df.to_csv(f'cars_data{year}.csv', index=False, encoding='utf-8-sig')

